# EMI Metal Detector Design

## Design constraints
Mostly dictated by motor board. 
The goal being to reuse the motor board with minimum modifications to speed up the development.

### Shunt resistor
- R: 10m$\Omega$

### Op amp (INA282) 
- Bandwidth: 10kHZ
- Gain: 50
- Gain error: +/- 1.4%

### ADC low pass
- Cut-off: 4.7kHz

## Coil design

### Coil time constant
A coil time constant is given by
$$\tau = \frac{L}{R}$$

Given limiting sampling frequency: 4.7kHz, by Shannon we can measure well up to 500Hz signals.

Optimal coil time constant is 
$$\tau_{optimal} \approx \frac{1}{500} \approx 2ms$$

### Cylindrical coil inductance
A cylindrical coil inductance is given by
$$L = \frac{r^2 N^2}{20r + 28d}$$

where 
- L is the inductance in $\mu$H, 
- r is the mean radius in cm, 
- N is the number of turns, 
- d is depth of the coil (outer radius minus inner radius) in cm.

Source: Spiral air coil formula https://en.wikipedia.org/wiki/Inductor#Inductance_formulas

Converted to SI units ($H$ and $m$):
$$L = 10^{-4} \frac{r^2 N^2}{20r + 28d}$$

### Cylidrical coil resistance
Resistance is defined as
$$R = \frac{\rho l}{A} = \alpha l$$
with $\alpha = \frac{\rho}{A}$ the wire linear resistivity in $\Omega$/m, where
- $\rho$ is the resistivity in $\Omega$m
- l the length of the wire in m
- A the cross section of the wire in m$^2$

For a cylindrical coil, the wire length is given by the product of the cylinder perimeter and the number of turns:
$$l = 2 \pi r N$$

Such that the resistance of a cylindrical coil is
$$R = 2 \pi \alpha r N $$

### Coil time constant simplification
Plugging the formulas from above in the RL circuit time constant we get
$$\tau = \frac{L}{R} = \frac{10^{-4}}{2\pi} \frac{r N}{\alpha (20r + 28d)}$$

For a big enough coil (r > 5cm), the depth of the coil $d$ is smaller than the coil radius $d << r$ so we can neglect in the formula, which yields the simplified version:
$$\tau \approx \frac{10^{-4}}{2\pi} \frac{r N}{20\alpha r} = \frac{10^{-5}}{4\pi} \frac{N}{\alpha}$$

The time constant now only depends on the number of turns $N$ and the linear resistivity of the wire $\alpha$.
These are our two design parameters to match the desired $\tau_{optimal}$.

### Fixing other parameters
The depth of the coil should be kept minimal (<1cm) given the simplification done above.

The radius of the coil determines what size of objects our detector will sense best.
We want to detect Anti-Personal landmine size objects which range from 6cm to 14cm. So our coil should be in 14-30cm diameter range, so $r \in [0.07, 0.15]$.

### Energy considerations
Energy stored in a coil is given by
$$W = \frac{1}{2} L I^2 = \frac{1}{2} \frac{L U^2}{R^2}$$

### Voltage detection constraints
Or antenna size https://em.geosci.xyz/content/geophysical_surveys/gpr/survey.html? Now minimal Voltage that could be detected is ???. How Voltage could be defined in the system:

### Decay of magnetic field (Probing Distance) 
As a general rule, the probing distance (D) is approximated 3 skin depths. If we assume the Earth is non-magnetic (μr=1):
https://em.geosci.xyz/content/geophysical_surveys/gpr/survey_design.html

### Scattering
Scattering is used to describe deviations in the paths of electromagnetic waves due to localized non-uniformities; which are less than 1/4 the wavelength of the radiowave signal. Scattering is problematic for GPR because it reduces the amplitudes of useful signals while increasing extraneous noise. It also make the interpretation of GPR data more challenging. Scattering noise could be reduced by using stacking (maybe 20 samples are enough as per example in the page). At this step, check 'Gain correction' section
https://em.geosci.xyz/content/geophysical_surveys/gpr/physics.html

### Ground penetration considerations
Ground penetration of the EM wave is characterized by the Beer Lambert law.
The order of magnitude is given by the skin depth
$$\delta = \sqrt{\frac{\rho}{\pi f \mu_r \mu_0}} \sqrt{\sqrt{1 + (2 \pi f \rho \epsilon_r \epsilon_0)^2} + 2 \pi f \rho \epsilon_r \epsilon_0}$$

with
- $\rho$ resistivity of the material
- $f$ the operating frequency
- $\mu_r$ the relative magnetic permeability
- $\mu_0$ the permeability of free space: $4 \pi \cdot 10^{-7} H/m$
- $\epsilon_r$ the relative permittivity of the material
- $\epsilon_0$ the permittivity of free space: $8.854187817 \cdot 10^{-12} F/m$

at this depth, the original wave amplitude $A_0$ is $\frac{A_0}{e} \approx 0.3679 A_0$

In [7]:
%matplotlib widget
from math import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

AWG_LINEAR_RESISTIVITY = {'0': 0.00032, '1': 0.0004, '2': 0.0005, '3': 0.00064, '4': 0.0008, '5': 0.001, '6': 0.00125, '7': 0.0016, '8': 0.002, '9': 0.0025, '10': 0.0032, '11': 0.004, '12': 0.005, '13': 0.0064, '14': 0.008, '15': 0.01, '16': 0.0125, '17': 0.016, '18': 0.02, '19': 0.025, '20': 0.032, '21': 0.04, '22': 0.05, '23': 0.064, '24': 0.08, '25': 0.1, '26': 0.125, '27': 0.16, '28': 0.2, '29': 0.25, '30': 0.32, '31': 0.4, '32': 0.5, '33': 0.64, '34': 0.8, '35': 1, '36': 1.25, '37': 1.6, '38': 2, '39': 2.5}
AWG_KG_PER_METER = {'0': 0.475, '1': 0.377, '2': 0.299, '3': 0.237, '4': 0.188, '5': 0.149, '6': 0.118, '7': 0.0938, '8': 0.0744, '9': 0.0590, '10': 0.0468, '11': 0.0371, '12': 0.0294, '13': 0.0234, '14': 0.0185, '15': 0.0147, '16': 0.0116, '17': 0.00922, '18': 0.00732, '19': 0.00580, '20': 0.00460, '21': 0.00365, '22': 0.00289, '23': 0.00229, '24': 0.00182, '25': 0.00144, '26': 0.00114, '27': 0.000908, '28': 0.000720, '29': 0.000571, '30': 0.000453, '31': 0.000359, '32': 0.000285, '33': 0.000226, '34': 0.000179, '35': 0.000142, '36': 0.000113, '37': 0.0000893, '38': 0.0000708, '39': 0.0000562}

In [9]:
def coil_inductance(r, N, d):
    return 1e-4 * ((r**2) * (N**2)) / (20 * r + 28 * d)

def coil_length(r, N):
    return 2 * pi * r * N

def coil_resistance(r, N, alpha):
    return coil_length(r, N) * alpha

def coil_turn_count(tau, alpha): 
    return (4 * pi) * (1e5) * tau * alpha

def coil_energy(L, R, U):
    return 0.5 * L * (U**2) / (R**2)

def coil_mass(r, N, awg):
    return coil_length(r, N) * AWG_KG_PER_METER[str(awg)]
    
def design_coil(tau, awg, r = 0.07, d = 0.01):
    alpha = AWG_LINEAR_RESISTIVITY[str(awg)]
    N = round(coil_turn_count(tau, alpha))
    L = coil_inductance(r, N, d)
    R = coil_resistance(r, N, alpha)
    return N, L, R, L/R, r

In [13]:
def show_coil(tau_ms, awg, r_cm = 7, d_cm = 1, voltage=12):
    N, L, R, tau, r = design_coil(tau_ms/1000, awg, r_cm/100, d_cm/100)
    print('Coil with     {} turns'.format(N))
    print('Inductance    {:.7f} H'.format(L))
    print('Resistance    {:.7f} Ohm'.format(R))
    print('Tau           {:.7f} s'.format(tau))
    print('Energy        {:.7f} J'.format(coil_energy(L, R, voltage)))
    print('You will need {:.7f} m of wire'.format(coil_length(r, N)))
    print('Coil mass     {:.7f} g'.format(coil_mass(r, N, awg) * 1000))

interact(show_coil, 
         tau_ms=widgets.BoundedFloatText(value=2.0, min=0.1, max=50.0, description='Tau [ms]'), 
         awg=widgets.Dropdown(options=AWG_LINEAR_RESISTIVITY.keys(), value='20', description='AWG'),
         r_cm=widgets.BoundedFloatText(value=7.0, min=1.0, max=30.0, description='Radius [cm]'),
         d_cm=widgets.BoundedFloatText(value=0.5, min=0.1, max=2.0, step=0.1, description='Depth [cm]'),
         voltage=widgets.BoundedFloatText(value=10.0, min=1.0, max=12.0, step=0.5, description='Voltage [V]'));

interactive(children=(BoundedFloatText(value=2.0, description='Tau [ms]', max=50.0, min=0.1), Dropdown(descrip…

In [11]:
rho = 100 # 1000
mu_0 = 4 * pi * 1e-7
mu_r = 1
mu = mu_0 * mu_r
eps_0 = 8.854187817e-12
eps_r = 25 # 4
eps = eps_0 * eps_r
tau = 2e-3
f = 1 / tau 
delta = sqrt(rho/(pi * f * mu)) * sqrt(sqrt(1 + (2 * pi * f * rho * eps)**2) + 2 * pi * f * rho * eps)
delta # skin depth

225.0869052456384